# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
cities = pd.read_csv("../output_data/weatherdata.csv", index_col = 0)
cities

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,72.34,76,22,8.59,PF,1658504306
1,avarua,-21.2078,-159.7750,78.85,73,20,12.66,CK,1658510063
2,cherskiy,68.7500,161.3000,51.89,49,18,2.73,RU,1658498247
3,hilo,19.7297,-155.0900,77.88,81,100,10.36,US,1658505151
4,rundu,-17.9333,19.7667,54.66,61,1,6.55,NaN,1658553042
...,...,...,...,...,...,...,...,...,...
535,bay roberts,47.5999,-53.2648,69.04,90,4,7.36,CA,1658476692
536,yavatmal,20.4000,78.1333,72.21,94,100,9.08,IN,1658535517
537,fort nelson,58.8053,-122.7002,75.70,32,38,6.06,CA,1658489910
538,dakar,14.6937,-17.4441,76.08,83,100,11.45,SN,1658559025


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
gmaps.configure(api_key=g_key)

In [41]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(cities[["Latitude", "Longitude"]], weights = cities["Humidity"]))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
weather_specfics = cities[(cities["Max Temp"] < 80) & (cities["Max Temp"] > 70) & (cities["Wind Speed"] < 10) & \
                          (cities["Cloudiness"] == 0)]
weather_specfics

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,albany,42.6001,-73.9662,75.49,69,0,3.56,US,1658482662
29,jamestown,42.0970,-79.2353,73.42,68,0,5.75,US,1658484018
30,yinchuan,38.4681,106.2731,77.27,22,0,5.93,CN,1658526521
34,santa cruz,-17.8000,-63.1667,72.86,73,0,6.91,BO,1658486545
57,sidi ali,36.0998,0.4206,73.15,82,0,1.79,DZ,1658552293
108,timizart,36.8000,4.2667,78.51,40,0,3.56,DZ,1658551268
152,manolas,38.0500,21.3500,70.84,76,0,0.00,GR,1658546978
158,alamosa,37.4695,-105.8700,78.78,34,0,9.22,US,1658491169
229,bethel,41.3712,-73.4140,75.38,86,0,0.00,US,1658482746
234,tyup,42.7276,78.3648,70.29,34,0,3.09,KG,1658532513


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = weather_specfics[:]
hotel_df["Hotel Name"] = ""

C:\Users\cabon\AppData\Local\Temp/ipykernel_10780/1826436776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [44]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    
    address = requests.get(url, params = params)
    lounge = address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = lounge["results"][0]["name"]
    except:
        print(f"Hotel not found near {row['City']}")
hotel_df.head()

Hotel not found near albany
Hotel not found near nalut


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,albany,42.6001,-73.9662,75.49,69,0,3.56,US,1658482662,
29,jamestown,42.0970,-79.2353,73.42,68,0,5.75,US,1658484018,Hampton Inn & Suites Jamestown
30,yinchuan,38.4681,106.2731,77.27,22,0,5.93,CN,1658526521,JJ Inns - Yinchuan New Moon Square Branch
34,santa cruz,-17.8000,-63.1667,72.86,73,0,6.91,BO,1658486545,Aviador Hotel Boutique
57,sidi ali,36.0998,0.4206,73.15,82,0,1.79,DZ,1658552293,Complexe touristique haje bezine


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [46]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))